In [1]:
import pandas as pd
import re
from textblob import TextBlob
from nltk.corpus import stopwords

chunksize = 100000
nrows = 500000
data_filename = "../Data/tweets.csv"
cols = ["timestamp", "text"]
textblobs = []

with pd.read_csv(data_filename, sep=';', chunksize=chunksize, nrows=500000, usecols=cols, lineterminator="\r") as reader:
    for chunk in reader:
        for i, row in chunk.iterrows():
            if type(row["text"]) == str and row["text"].isascii():
                textblobs.append(TextBlob(row["text"].lower()))

In [2]:
for i in range(len(textblobs)):
    words_without_symbols = []
    for word in textblobs[i].words:
        cleaned_word = re.sub("^[^a-z]+$", "", word)
        cleaned_word = re.sub("http", "", cleaned_word)
        cleaned_ord = re.sub("https", "", cleaned_word)
        if cleaned_word != "":
            words_without_symbols.append(cleaned_word)
    without_stop_words = [word for word in words_without_symbols if not word in stopwords.words()]
    textblobs[i] = TextBlob(" ".join(without_stop_words))
            
for i in range(20):
    print(textblobs[i].words)

['another', 'test', 'tweet', "n't", 'caught', 'stream', 'bitcoin']
['current', 'crypto', 'prices', 'btc', 'usd', 'eth', 'usd', 'ltc', 'usd', 'bch', 'usd', 'xlm', 'usd', 'doge', 'usd', 'neo', 'usd', 'xrp', 'usd', 'cann', 'usd', 'emc2', 'usd', 'xmr', 'usd', 'btg', 'usd']
['spiv', 'nosar', 'baz', 'bitcoin', 'asset', 'amp', 'currency', 't.co/fjru1ooxsm', 't.co/fybx4h6r6r', 't.co/xzsjmz0h3k', 't.co/v7grfiqg7w', 't.co/oq5hqt5hnn']
['nwoodfine', 'building', 'real', 'bitcoin', 'sv', 'building', 'broken', 'btc']
['change', 'coming', 'get', 'ready', 'boom', 'another', 'cb', 'jab', 'nothing', 'stop', 'globalism', 'stage', 'cb', 'push', 'make', 'world', 'govt', 'coming', 'people', 'taking', 'lead', 'amp', 'leaders', 'follow', 'majority', 'wwg1wgq', 't.co/tajfwxnwd4']
['useful', 'articles', 'stefan', 'guide', 'run', 'ltonetwork', 'node', 'alibaba', 'cloud', 't.co/ij9rlkaabt', 'ltonetwork', 'lto', 'eth', 'xrpcommmunity', 'crypto', 'xlm', 'xrp', 'blockchain', 'xrpcommmunity', 'eos', 'xmr', 'trx', 'lt